# Project description

This project is part of Udacity PyTorch Scholarship Challenge. More details on the scholarship could be find here - https://www.udacity.com/course/deep-learning-pytorch--ud188

In this project, the task is to train an image classifier to recognize different species of flowers. Detailed dataset description is here - http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html. It consits of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content


# Google Colab Environment Setting

Because of the complex network arhitecture, training on the CPU might take ages. The quickest and cheapest option is to use Google's new tool - Google CoLab. It provides Tesla K80 GPU 12GB for each Google account for free. In order to install all dependencies for CoLab environment, run the next cell.

As an alternative, one might use any GPU with CUDA support locally or die hard and train on the CPU. In this case, ignore the next cell and jump to the imports section. 

In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

!pip install Pillow==4.1.1
!pip install PIL
!pip install image


# workaround 
from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
    for extension in extensions: 
        register_extension(id, extension)
Image.register_extensions = register_extensions

#Mounting Google Drive
from google.colab import drive
drive.mount('/gdrive')

#Moving to the flowers directory (might be different for your file structure)
%cd ../
%cd 'gdrive/My Drive/Flowers'
%cd 'Flowers'

In [ ]:
# Loading dependencies for the project 
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
%matplotlib inline

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, one needs to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. We would use pre-trained network (transfer learning), thus, the input data should be resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. The only transformations needed is that each image should be cropped to 224x224 pixels.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets we need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [1]:
#Data path (might be different for your file structure)
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [ ]:
# Defining transforms for the training and validation sets
train_transforms = transforms.Compose([transforms.RandomRotation(15), 
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(p=0.1),
                                      transforms.ToTensor(), 
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Loading the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)

### Label mapping

Category label is mapped to category name in the JSON file - `cat_to_name.json`. This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

The easiest way to complete this task is to use a pre-trained models from `torchvision.models` to get the image features. 

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

Provided example uses ResNet50 model, which gives you "7.13%" error for top-5 classes. In other words, the model will correctly predict "92.87%" of the image classification task. 



In [ ]:
#Loading pre-trained model
model = models.resnet50(pretrained=True)
model

In [ ]:
# Freeze parameters. This is important step. All parameters for features should be freezed. 
#Otherwise, the model will be changing gradients of features at each epoch. 
for param in model.parameters():
    param.requires_grad = False
    
#Loading new layers to the trained model
#This is the layers I used for the training. It couldbe changed in different ways.
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 512)),
                          ('relu', nn.ReLU()),
                          ('drop', nn.Dropout(p=0.5)),
                          ('fc2', nn.Linear(512, 256)),
                          ('relu', nn.ReLU()),
                          ('drop', nn.Dropout(p=0.5)),
                          ('fc3', nn.Linear(256, 102)),
                          ]))
#Adding your layer to the model as the fully-connected layer    
model.fc = classifier
#model

In [ ]:
# specify loss function (I used categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (needed to compute backward propagation)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# Use GPU if it's available (if CUDA is available, run on CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

In [ ]:
#Defining your checkpoint information. Checkpoint is used to save your model during each epoch.
model.class_to_idx = train_data.class_to_idx
checkpoint = OrderedDict({'input_size': 2048,
              'output_size': 102,
              'epoch': 0,
              'classifier':classifier,
              'ind': model.class_to_idx,
              'state_dict': model.state_dict(),
              'optim': optimizer.state_dict()})

In [ ]:
#Training part. This cell will run the model through n-number of epochs. Each epoch will print the following statistics:
#Epoch -- Training loss -- Validation loss -- Validation accuracy -- Duration of each epoch

from datetime import datetime
# number of epochs to train the model
n_epochs = 25

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):
    start_time = datetime.now()
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    # keep track of accuracy for the validation set
    total = 0
    correct = 0
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in trainloader:
        # enable CUDA
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in validloader: 
        # enable CUDA
        data, target = data.to(device), target.to(device)
        # forward pass
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(validloader.dataset)
    
    #calculate average accuracy
    _, top_class = torch.max(output, 1)
    total += target.size(0)
    correct += (top_class == target).sum().item()
    end_time = datetime.now() 
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {} \tDuration: {}'.format(epoch, train_loss, valid_loss, (correct/total),(end_time - start_time)))
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        checkpoint['state_dict'] = model.state_dict()
        checkpoint['optim'] = optimizer.state_dict()
        checkpoint['epoch'] = epoch
        torch.save(checkpoint, 'model_flowers2.pt')
        valid_loss_min = valid_loss